In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import Audio as Play
from pyannote.database import get_protocol, FileFinder
from pyannote.audio.utils.preview import listen
from pyannote.audio.core.io import Audio
from pyannote.core import Segment

In [ ]:
# gett a 5s excerpt of first test file
protocol = get_protocol('Debug.SpeakerDiarization.Debug', 
                        preprocessors={"audio": FileFinder()})
file = next(protocol.test())

audio = Audio(sample_rate=16000, mono=True)

# Load cropped audio
waveform, sample_rate = audio.crop(file, Segment(0, 5))

# Listen
print(waveform.shape, sample_rate)
listen({'waveform':waveform, 'sample_rate': sample_rate})

## Augmentation Pipeline

In [ ]:
from pyannote.audio.transforms.pipeline import TransformsPipe

from pyannote.audio.tasks.segmentation.voice_activity_detection import VoiceActivityDetection
from pyannote.audio.models.debug import SimpleSegmentationModel
vad = VoiceActivityDetection(protocol, batch_size=2)
model = SimpleSegmentationModel(task=vad)

In [ ]:
from pyannote.audio.transforms.transforms import Reverb
reverb = Reverb(p=1, sample_rate=16000)(waveform[None].repeat(3,1,1))
Play(reverb[1], rate=16000)

In [ ]:
# define a model that simply returns the waveform
from pyannote.audio.core.model import Model
class Passthrough(Model):
    def forward(self, waveforms):
        return waveforms
    
identity = Passthrough()

In [ ]:
# pass the waveform through this "identity" model
Play(identity(waveform).squeeze(), rate=sample_rate, normalize=False, autoplay=True)

In [ ]:
# add one torch_audiomentations waveform transform to the model
from pyannote.audio.augmentation.registry import register_augmentation
from torch_audiomentations import Gain
gain = Gain(
    min_gain_in_db=-15.0,
    max_gain_in_db=5.0,
    p=0.5)
register_augmentation(gain, identity, when='input')

In [ ]:
identity

In [ ]:
# pass the waveform through the "augmented" model
Play(identity(waveform).squeeze(), rate=sample_rate, normalize=False, autoplay=True)